# Question 2 Workflow


## Workfolow Summary

## Imports

In [1]:
# Import pandas and biothings explorers modules
import pandas as pandas
from biothings_explorer.query.predict import Predict
from biothings_explorer.query.visualize import display_graph
from biothings_explorer.user_query_dispatcher import FindConnection
from biothings_explorer.hint import Hint
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline
import warnings
warnings.filterwarnings("ignore") 

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/data_tools/df_processing.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Set Parameters

In [2]:
max_direct = 20
max_indirect = 50

In [3]:
disease_name = 'COVID-19'
ht = Hint()
disease = ht.query(disease_name)['Disease'][0]
disease

{'MONDO': 'MONDO:0100096',
 'DOID': 'DOID:0080600',
 'name': 'COVID-19',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0100096'},
 'display': 'MONDO(MONDO:0100096) DOID(DOID:0080600) name(COVID-19)',
 'type': 'Disease'}

## Get Symptoms

In [1]:
import requests 

symptoms = ([ 'fever','cough', 'hemoptysis', 
'shortness of breath', 'myalgia', 'fatigue', 'sore throat', 
'nausea', 'vomiting', 'diarrhea', 'conjunctivitis', 'anorexia',
'headache', 'leukopenia', 'eosinopenia', 'neutrophilia', 'elevated liver enzymes', 'C-reactive protein', 'ferritin',
'hyaline membrane', 'alveolar lesions', 'prominent hyaline membranes', 'alveolar damage',
 'acute myocardial injury', ' myocarditis', 'arrhythmias','cardiac dysfunction', 'encephalopathy'])


symptom_dict = {}

for symptom_name in symptoms: 
    try: 
        symptom = ht.query(symptom_name)['PhenotypicFeature'][0]
#         print(symptom['name'])
        pd = Predict(
            input_objs=[symptom],
            intermediate_nodes =[None], 
            output_types = ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'], 
            config={"expand": False}
        )
        pd.connect(verbose=False)
        df = pd.display_table_view()
        edges_out = df.shape[0]
        symptom_dict[symptom['HP']] = {
            'names': [symptom['name'].upper()],
            'frequency': 'Unknown', 
            'edges_out' : edges_out
        }
        r = requests.get('https://biothings.ncats.io/hpo/phenotype/' + str(symptom['HP']))
        res = r.json()
#         print(res)
        if('synonym' in res):
            if('exact' in res['synonym']):
                for name in res['synonym']['exact']:
                    if name.upper() not in symptom_dict[res['_id']]['names']: 
#                         print(name.upper())
                        symptom_dict[res['_id']]['names'].append(name.upper())
    except:
        print(symptom_name + " failed")


In [5]:
# symptom_dict
disease_symptom_df = pandas.DataFrame.from_dict(symptom_dict, orient='index')
disease_symptom_df

,names,frequency,edges_out
HP:0011134,"[LOW-GRADE FEVER, MILD FEVER]",Unknown,9553
HP:0002105,"[HEMOPTYSIS, COUGHING UP BLOOD, HAEMOPTYSIS]",Unknown,46624
HP:0003326,"[MYALGIA, MUSCLE ACHE, MUSCLE PAIN]",Unknown,116332
HP:0500039,"[CONJUNCTIVAL CICATRIZATION, CICATRICIAL CONJU...",Unknown,3
HP:0002039,[ANOREXIA],Unknown,125701
HP:0011897,"[NEUTROPHILIA, INCREASED BLOOD NEUTROPHIL COUNTS]",Unknown,6354
HP:0032436,[ABNORMAL C-REACTIVE PROTEIN LEVEL],Unknown,257
HP:0040133,"[ABNORMAL SERUM FERRITIN, ABNORMAL PLASMA FERR...",Unknown,440
HP:0005115,"[SUPRAVENTRICULAR ARRHYTHMIA, ARRHYTHMIAS, SUP...",Unknown,11784


In [6]:
## also save disease symptoms as a list
disease_symptoms = []
for x in disease_symptom_df['names']: 
    disease_symptoms = disease_symptoms + x
# disease_symptoms

## Disease -> Gene -> [Phenotypic Feature, Disease, Biological Process]

In [7]:
pd = Predict(
    input_objs=[disease],
    intermediate_nodes =['Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config={"expand": False}
)
pd.connect(verbose=False)

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 165, in callSingleAPI
    async with session.get(
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aiohttp/client.py", line 1012, in __aenter__
    self._resp = await self._coro
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aiohttp/client.py", line 504, in _request
    await resp.start(conn)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aiohttp/client_reqrep.py", line 847, in start
    message, payload = await self._protocol.read()  # type: ignore  # noqa
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aiohttp/streams.py", line 591, in read
    await self._waiter
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/asyncio/futures.py", line 260, in 

API call to BioLink API with input 128989 failed with unknown response
API call to BioLink API with input R-MMU-8949664 failed with unknown response
API call to BioLink API with input HP:0002448 failed with unknown response
API call to BioLink API with input HP:0002480 failed with unknown response
API call to Automat CORD19 Scigraph API with input 1401 failed with timeout error. Current timeout limit is 5 seconds
API call to Automat CORD19 Scigraph API with input 59272 failed with timeout error. Current timeout limit is 5 seconds
API call to Automat PHAROS API with input 6651 failed with timeout error. Current timeout limit is 5 seconds
API call to Automat CORD19 Scibite API with input 59272 failed with timeout error. Current timeout limit is 5 seconds
API call to Automat CORD19 Scibite API with input 7113 failed with timeout error. Current timeout limit is 5 seconds
API call to Automat PHAROS API with input 1401 failed with timeout error. Current timeout limit is 5 seconds
API call to

In [8]:
df = pd.display_table_view()
df.shape

(34593, 19)

In [9]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df['output_label'])]) if val in disease_symptoms]
direct_relevant_genes_to_symptoms_df = df.iloc[indices_with_symptom_outputs]
direct_relevant_genes_to_symptoms_df.shape

(9, 19)

In [10]:
direct_genes = 	list(direct_relevant_genes_to_symptoms_df["node1_label"])
direct_gene_dict = {x:direct_genes.count(x) for x in direct_genes}
direct_gene_dict = dict(sorted(direct_gene_dict.items(), key=lambda x: x[1], reverse=True)[:max_direct])
direct_gene_dict

{'TNF': 4, 'IL2': 1, 'IL1B': 1, 'IFNA1': 1, 'TNNI3': 1, 'IL6': 1}

## Disease X -> Intermediate Node -> Gene ->  [Phenotypic Feature, Disease, Biological Process]

In [11]:
query_config = {
    "filters": [
        {
        },
        {
            "nodeDegree": {
                ">" : 100
            }
        }
    ],
    "annotate": ["ngd","nodeDegree"],
    "expand": False
}

In [14]:
## way to filter at the gene step for top x number of genes? 
pd2 = Predict(
    input_objs=[disease],
    intermediate_nodes =[['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 
                'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 
                'Pathway', 'AnatomicalEntity', 'PhenotypicFeature'],'Gene'], 
    output_types =['PhenotypicFeature', 'Disease', 'BiologicalProcess'], 
    config= query_config 
)
pd2.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

Your query have 1 input nodes, including COVID-19 .... And BTE will find paths that connect your input nodes to your output types ['PhenotypicFeature', 'Disease', 'BiologicalProcess']. Paths will contain 2 intermediate nodes.

Intermediate node #1 will have these type constraints: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Intermediate node #2 will have these type constraints: Gene

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Disease
Output Types: ['Gene', 'SequenceVariant', 'ChemicalSubstance', 'Disease', 'MolecularActivity', 'BiologicalProcess', 'CellularComponent', 'Pathway', 'AnatomicalEntity', 'PhenotypicFeature']
Predicates: None

BTE found 12 APIs based on SmartAPI Meta-KG.

API 1. Ontology Lookup Service API (1 API calls)
API 2. Automat CORD19 Scibite API (3 AP

API 15.1 MyChem.info API: 31 hits
API 16.1: http://ctdbase.org/tools/batchQuery.go?format=json&inputTerms=D019438&inputType=chem&report=genes_curated
API 4.1: https://biothings.ncats.io/tcga_mut_freq_kp/query?fields=association.freq_by_sample,subject.SYMBOL&q=object.id:"MONDO:0020753"%20AND%20association.freq_by_sample:>0.1&size=1000&sort=-association.freq_by_sample
API 4.1 TCGA Mutation Frequency KP API: 0 hits
API 3.1: https://pending.biothings.io/mgigene2phenotype/query?fields=_id&size=300 (POST -d q=DOID:0080599&scopes=mgi.associated_with_disease.doid)
API 3.1 MGIgene2phenotype API: 0 hits
API 6.1: https://pending.biothings.io/DISEASES/query?fields=DISEASES.associatedWith (POST -d q=DOID:0080599&scopes=DISEASES.doid)
API 6.1 DISEASES API: 0 hits
API 9.1: https://biothings.ncats.io/cord_disease/query?fields=associated_with (POST -d q=DOID:0080599&scopes=doid)
API call to CORD Disease API with input ['DOID:0080599'] failed with unknown response
API 15.2: https://mychem.info/v1/query?

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API 15.2 MyChem.info API: 21 hits
API 16.1 CTD API: 127 hits
API 1.1: https://automat.renci.org/cord19-scibite/disease/gene/MONDO:0020753
API 1.1 Automat CORD19 Scibite API: 0 hits
API 19.1: https://automat.renci.org/hmdb/chemical_substance/gene/CHEBI:33098
API 19.1 Automat HMDB API: 0 hits
API 1.2: https://automat.renci.org/cord19-scibite/chemical_substance/gene/CHEBI:134730
API 1.2 Automat CORD19 Scibite API: 0 hits
API 18.1: https://automat.renci.org/chembio/chemical_substance/gene/CHEBI:45409
API 18.1 Automat CHEMBIO API: 0 hits
API 2.1: https://automat.renci.org/cord19-scigraph/chemical_substance/gene/CHEBI:5801
API 2.1 Automat CORD19 Scigraph API: 2 hits
API 2.2: https://automat.renci.org/cord19-scigraph/chemical_substance/gene/CHEBI:134730
API 2.2 Automat CORD19 Scigraph API: 0 hits
API 2.3: https://automat.renci.org/cord19-scigraph/disease/gene/MONDO:0020753
API 2.3 Automat CORD19 Scigraph API: 0 hits
API 5.1: https://automat.renci.org/pharos/chemical_substance/gene/CHEBI:13473

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


...

API 7.27 BioLink API: 74 hits
API 7.28: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2147/interactions?direct=True&rows=200&unselect_evidence=True
API 7.28 BioLink API: 200 hits
API 7.29: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2580/interactions?direct=True&rows=200&unselect_evidence=True
API 7.29 BioLink API: 194 hits
API 13.30: https://biothings.ncats.io/text_mining_targeted_association/query?fields=object.NCBIgene,association.evidence.publications&q=subject.CHEBI:"CHEBI:31781" AND association.edge_label:negatively_regulates_entity_to_entity&size=1000
API 13.30 Text Mining Targeted Association API: 0 hits
API 13.31: https://biothings.ncats.io/text_mining_targeted_association/query?fields=object.NCBIgene,association.evidence.publications&q=subject.CHEBI:"CHEBI:132943" AND association.edge_label:negatively_regulates_entity_to_entity&size=1000
API 13.31 Text Mining Targeted Association API: 0 hits
API 13.32: https://biothings.ncats.io/text_mining_t

API 7.72 BioLink API: 0 hits
API 7.73: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3574/interactions?direct=True&rows=200&unselect_evidence=True
API 7.73 BioLink API: 0 hits
API 7.74: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:1636/interactions?direct=True&rows=200&unselect_evidence=True
API 7.74 BioLink API: 0 hits
API 7.75: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:340024/interactions?direct=True&rows=200&unselect_evidence=True
API 7.75 BioLink API: 0 hits
API 7.76: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3627/interactions?direct=True&rows=200&unselect_evidence=True
API 7.76 BioLink API: 0 hits
API 7.77: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3576/interactions?direct=True&rows=200&unselect_evidence=True
API 7.77 BioLink API: 0 hits
API 7.78: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:2147/interactions?direct=True&rows=200&unselect_evidence=True
API 7.78 BioLink API

API 9.1 CTD API: 178 hits
API 5.1: https://biothings.ncats.io/cord_gene/query?fields=associated_with (POST -d q=11892&scopes=hgnc)
API 10.1: https://translator.broadinstitute.org/molepro_reasoner/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'related_to'}], 'nodes': [{'curie': 'HGNC:11892', 'id': 'n00', 'type': 'gene'}, {'id': 'n01', 'type': 'disease'}]}}})
API 10.1 Molecular Data Provider API: 0 hits
API 11.1: https://translator.broadinstitute.org/genetics_data_provider/query (POST -d {'message': {'query_graph': {'edges': [{'id': 'e00', 'source_id': 'n00', 'target_id': 'n01', 'type': 'associated'}], 'nodes': [{'curie': 'NCBIGene:7124', 'id': 'n00', 'type': 'gene'}, {'id': 'n01', 'type': 'disease'}]}}})
API 8.2: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:7124/phenotypes?direct=True&rows=200&unselect_evidence=True
API 8.2 BioLink API: 0 hits
API 7.1: https://pending.biothings.io/DISEASES/query?field

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Ver

API 13.2 SEMMED Gene API: 1681 hits
API 13.3 SEMMED Gene API: 6 hits
API 13.4 SEMMED Gene API: 2307 hits
API 13.5: https://biothings.ncats.io/semmedgene/query?fields=disrupted_by (POST -d q=C0017337,C0812246&scopes=umls)
API 13.5 SEMMED Gene API: 0 hits
API 13.6: https://biothings.ncats.io/semmedgene/query?fields=disrupted_by (POST -d q=C0017337,C0812246&scopes=umls)
API 13.6 SEMMED Gene API: 0 hits
API 13.7: https://biothings.ncats.io/semmedgene/query?fields=causes (POST -d q=C0017337,C0812246&scopes=umls)
API 13.7: https://biothings.ncats.io/semmedgene/query?fields=causes (POST -d q=C0017337,C0812246&scopes=umls)
API 13.7: https://biothings.ncats.io/semmedgene/query?fields=affects (POST -d q=C0017337,C0812246&scopes=umls)
API 13.7 SEMMED Gene API: 360 hits
API 13.8 SEMMED Gene API: 2390 hits
API 13.9 SEMMED Gene API: 1 hits
API 13.10: https://biothings.ncats.io/semmedgene/query?fields=disrupts (POST -d q=C0017337,C0812246&scopes=umls)
API 13.10: https://biothings.ncats.io/semmedgene/

In [15]:
df2 = pd2.display_table_view()
df2.shape

(1905047, 27)

In [16]:
# filter gene -> symptoms table to only include rows where symptoms match disease symptoms
indices_with_symptom_outputs = [i for i, val in enumerate([x.upper() for x in list(df2['output_label'])]) if val in disease_symptoms]
indirect_relevant_genes_to_symptoms_df = df2.iloc[indices_with_symptom_outputs]
indirect_relevant_genes_to_symptoms_df.shape

(806, 27)

In [17]:
indirect_genes = 	list(indirect_relevant_genes_to_symptoms_df["node1_label"])
indirect_gene_dict = {x:indirect_genes.count(x) for x in indirect_genes}
indirect_gene_dict = dict(sorted(indirect_gene_dict.items(), key=lambda x: x[1], reverse=True)[:max_direct])
indirect_gene_dict

{'dioxygen': 60,
 'chloroquine': 54,
 'REN': 52,
 'IL2': 34,
 'IFNA1': 34,
 'IL6': 34,
 'CCL2': 32,
 'CD4': 30,
 'TNF': 30,
 'IL10': 30,
 'ALB': 28,
 'IL2RA': 28,
 'CCL3': 24,
 'hydroxychloroquine': 24,
 'ACE': 20,
 'CXCL8': 20,
 'TH': 18,
 'IL1B': 18,
 'IFNB1': 18,
 'AGTR1': 16}

## Top Genes -> Pathways 

In [18]:
# get gene inputs through hint module
gene_inputs = []
for gene in list(direct_gene_dict.keys()): 
    try: 
        gene_input = ht.query(gene)["Gene"][0]
        gene_inputs.append(gene_input)
    except: 
        print(gene + ' Failed')

# print(gene_inputs)

In [19]:
pd3 = Predict(
    input_objs=gene_inputs,
    intermediate_nodes=[None], 
    output_types =['Pathway'], 
    config={"expand": False}
)
pd3.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

Your query have 6 input nodes, including tumor necrosis factor,interleukin 2,interleukin 1 beta,interferon alpha 1,troponin I3, cardiac type .... And BTE will find paths that connect your input nodes to your output types ['Pathway']. Paths will contain 1 intermediate nodes.

Intermediate node #1 will have these type constraints: None

========== Query # 1 ==========

==== Step #1: Query Path Planning ====

Input Types: Gene
Output Types: None
Predicates: None

BTE found 21 APIs based on SmartAPI Meta-KG.

API 1. Text Mining Targeted Association API (12 API calls)
API 2. Automat CORD19 Scibite API (24 API calls)
API 3. mydisease.info API (1 API calls)
API 4. Automat CORD19 Scigraph API (24 API calls)
API 5. TCGA Mutation Frequency KP API (1 API calls)
API 6. CORD Gene API (11 API calls)
API 7. Automat PHAROS API (24 API calls)
API 8. MyChem.info API (3 API calls)
API 9. DISEASES API (1 API calls)
API 10. BioLink API (30 API calls)
API 11. O

API 10.2 BioLink API: 20 hits
API 3.1: http://mydisease.info/v1/query?fields=disgenet.xrefs.umls&size=250 (POST -d q=7124,3553,3439,3569,7137,3558&scopes=disgenet.genes_related_to_disease.gene_id)
API 3.1 mydisease.info API: 582 hits
API 10.3: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3439/anatomy?direct=True&rows=200&unselect_evidence=True
API 10.3 BioLink API: 19 hits
API 10.4: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3558/anatomy?direct=True&rows=200&unselect_evidence=True
API 10.4 BioLink API: 20 hits
API 6.1: https://biothings.ncats.io/cord_gene/query?fields=associated_with (POST -d q=11892,5992,6001,5417,11947,6018&scopes=hgnc)
API 19.1: https://pending.biothings.io/ebigene2phenotype/query?fields=gene2phenotype (POST -d q=11892,5992,6001,5417,11947,6018&scopes=_id)
API 19.1 EBIgene2phenotype API: 0 hits
API 12.4: http://ctdbase.org/tools/batchQuery.go?format=json&inputTerms=7124&inputType=gene&report=diseases_curated
API 12.4 CTD API: 178 

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Ver

API call to CORD Gene API with input ['11892', '5992', '6001', '5417', '11947', '6018'] failed with unknown response
API call to CORD Gene API with input ['11892', '5992', '6001', '5417', '11947', '6018'] failed with unknown response


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Ver

...

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/__init__.py", line 297, in callSingleAPI
    result = tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/__init__.py", line 49, in transform
    return self.tf.transform()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/base_transformer.py", line 78, in transform
    item = self.wrap(item)
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/biothings_explorer/call_apis/api_response_transform/transformers/cord_transformer.py", line 16, in wrap
    for k in item.keys():
RuntimeError: dictionary keys changed during iteration


API call to CORD Gene API with input ['11892', '5992', '6001', '5417', '11947', '6018'] failed with unknown response
API 18.15: https://biothings.ncats.io/semmedgene/query?fields=disrupted_by (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.15 SEMMED Gene API: 0 hits
API 7.15: https://automat.renci.org/pharos/gene/disease/NCBIGene:7124
API 7.15 Automat PHAROS API: 3 hits
API 7.16: https://automat.renci.org/pharos/gene/disease/NCBIGene:7137
API 7.16 Automat PHAROS API: 0 hits
API 4.15: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:7137
API 4.15 Automat CORD19 Scigraph API: 0 hits
API 7.17: https://automat.renci.org/pharos/gene/disease/NCBIGene:3439
API 7.17 Automat PHAROS API: 0 hits
API 4.16: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:3553
API 4.16 Automat CORD19 Scigraph API: 0 hits
API 4.17: https://automat.renci.org/cord19-scigraph/gene/disease/NCBIGene:3439
API 4.17 Automat CORD19 Scigraph API: 0 hits
API 7.

API 18.25 SEMMED Gene API: 1 hits
API 18.26 SEMMED Gene API: 18 hits
API 18.27 SEMMED Gene API: 2 hits
API 18.28 SEMMED Gene API: 0 hits
API 18.29 SEMMED Gene API: 0 hits
API 10.30: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3569/phenotypes?direct=True&rows=200&unselect_evidence=True
API 10.30 BioLink API: 0 hits
API 18.30: https://biothings.ncats.io/semmedgene/query?fields=negatively_regulated_by (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.30: https://biothings.ncats.io/semmedgene/query?fields=negatively_regulated_by (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.30: https://biothings.ncats.io/semmedgene/query?fields=negatively_regulated_by (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.30: https://biothings.ncats.io/semmedgene/query?fields=negatively_regulated_by (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.30:

API 18.70: https://biothings.ncats.io/semmedgene/query?fields=related_to (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.70: https://biothings.ncats.io/semmedgene/query?fields=treats (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.70: https://biothings.ncats.io/semmedgene/query?fields=related_to (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.70: https://biothings.ncats.io/semmedgene/query?fields=related_to (POST -d q=C1415900,C1334112,C1420826,C1334122,C0812246,C0879590&scopes=umls)
API 18.70 SEMMED Gene API: 0 hits
API 18.71 SEMMED Gene API: 2483 hits
API 18.72 SEMMED Gene API: 1309 hits
API 18.73 SEMMED Gene API: 0 hits
API 18.74 SEMMED Gene API: 4 hits

BTE starts to perform id-to-object translation.



KeyError: 'Transcript'

In [ ]:
df3 = pd3.display_table_view()
df3.shape